In [12]:
import time
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

from manual_queries import get_manual_tests, manual_test_names
from automatic_queries import get_auto_tests, auto_test_names

In [13]:
load_dotenv("./.env")

user = os.getenv("USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
database = os.getenv("DATABASE")

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")

In [ ]:
# Benchmark settings
limits = [5, 10, 20, 50, 100]
results = []

for limit in limits:
    manual_queries = get_manual_tests(limit)
    auto_queries = get_auto_tests(limit)

    # Run manual queries
    i = 1
    for query, name in zip(manual_queries, manual_test_names):
        start = time.time()
        try:
            df = pd.read_sql_query(query, engine)

            duration = time.time() - start
            results.append({
                "query_type": "manual",
                "test_name": name,
                "tables": i,
                "limit": limit,
                "duration_sec": round(duration, 4)
            })
        except Exception as e:
            results.append({
                "query_type": "manual",
                "test_name": name,
                "limit": limit,
                "duration_sec": None,
                "error": str(e)
            })
        
        i += 1
        print(f"Ran {name} with LIMIT {limit} in {duration:.4f} sec")

    # Run automatic queries
    i = 1
    for query, name in zip(auto_queries, auto_test_names):
        start = time.time()
        try:
            df = pd.read_sql_query(query, engine)

            duration = time.time() - start
            results.append({
                "query_type": "auto",
                "test_name": name,
                "tables": i,
                "limit": limit,
                "duration_sec": round(duration, 4)
            })
        except Exception as e:
            results.append({
                "query_type": "auto",
                "test_name": name,
                "limit": limit,
                "duration_sec": None,
                "error": str(e)
            })

        i += 1
        print(f"Ran {name} with LIMIT {limit} in {duration:.4f} sec")

    df_results = pd.DataFrame(results)
    print(df_results.head())
    # Save results to CSV
    df_results.to_csv(f"benchmark_results_limit_{limit}.csv", index=False)


Ran manual_test_agg_all_3 with LIMIT 5 in 3.1062 sec
Ran manual_test_agg_6 with LIMIT 5 in 0.0472 sec
Ran manual_test_where_having_1 with LIMIT 5 in 0.0442 sec
Ran manual_test_large_query_3 with LIMIT 5 in 0.0431 sec
Ran auto_test_agg_all_3 with LIMIT 5 in 4.1126 sec
Ran auto_test_agg_6 with LIMIT 5 in 0.0491 sec
Ran auto_test_where_having_1 with LIMIT 5 in 0.0456 sec
Ran auto_test_large_query_3 with LIMIT 5 in 0.0458 sec
  query_type                   test_name  tables  limit  duration_sec
0     manual       manual_test_agg_all_3       1      5        3.1062
1     manual           manual_test_agg_6       2      5        0.0472
2     manual  manual_test_where_having_1       3      5        0.0442
3     manual   manual_test_large_query_3       4      5        0.0431
4       auto         auto_test_agg_all_3       1      5        4.1126


In [11]:
df = pd.read_csv("benchmark_results_limit_5_20.csv")
df

,query_type,test_name,tables,limit,duration_sec
0,manual,manual_test_agg_all_3,1,5,3.1062
1,manual,manual_test_agg_6,2,5,0.0472
2,manual,manual_test_where_having_1,3,5,0.0442
3,manual,manual_test_large_query_3,4,5,0.0431
4,auto,auto_test_agg_all_3,1,5,4.1126
5,auto,auto_test_agg_6,2,5,0.0491
6,auto,auto_test_where_having_1,3,5,0.0456
7,auto,auto_test_large_query_3,4,5,0.0458
8,manual,manual_test_agg_all_3,1,10,70.4552
9,manual,manual_test_agg_6,2,10,0.4706
